In [1]:
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile
import urllib.request
from distutils.version import StrictVersion
from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image

import cv2
import time

#url = http://192.168.1.7/cgi-bin/snapshot.cgi

#cap = cv2.VideoCapture(0)
#cap.open ("http://192.168.1.7/cgi-bin/snapshot.cgi")
#cap = cv2.VideoCapture("http://192.168.1.7/cgi-bin/snapshot.cgi")
url = ("http://192.168.1.7/cgi-bin/snapshot.cgi")
#url = ("http://192.168.1.9:8080/shot.jpg")

#cap = cv2.imread("/home/aditya/Music/img5.jpg")
#cap.release()
# This is needed since the notebook is stored in the object_detection folder.
sys.path.append("..")
from object_detection.utils import ops as utils_ops

if StrictVersion(tf.__version__) < StrictVersion('1.12.0'):
  raise ImportError('Please upgrade your TensorFlow installation to v1.12.*.')


In [2]:
from utils import label_map_util

from utils import visualization_utils as vis_util

In [3]:
# What model to download.
MODEL_NAME = 'person4_inference_graph'

# Path to frozen detection graph. This is the actual model that is used for the object detection.
PATH_TO_FROZEN_GRAPH = MODEL_NAME + '/frozen_inference_graph.pb'

# List of the strings that is used to add correct label for each box.
PATH_TO_LABELS = os.path.join('training4', 'people-detection.pbtxt')
NUM_CLASSES = 1

In [4]:
detection_graph = tf.Graph()
with detection_graph.as_default():
  od_graph_def = tf.GraphDef()
  with tf.gfile.GFile(PATH_TO_FROZEN_GRAPH, 'rb') as fid:
    serialized_graph = fid.read()
    od_graph_def.ParseFromString(serialized_graph)
    tf.import_graph_def(od_graph_def, name='')

In [5]:
category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS, use_display_name=True)

In [6]:
def load_image_into_numpy_array(image):
  (im_width, im_height) = image.size
  return np.array(image.getdata()).reshape(
      (im_height, im_width, 3)).astype(np.uint8)

In [7]:
# For the sake of simplicity we will use only 2 images:
# image1.jpg
# image2.jpg
# If you want to test the code with your images, just add path to the images to the TEST_IMAGE_PATHS.
#PATH_TO_TEST_IMAGES_DIR = 'test_images'
#TEST_IMAGE_PATHS = [ os.path.join(PATH_TO_TEST_IMAGES_DIR, 'image{}.jpg'.format(i)) for i in range(3, 17) ]

# Size, in inches, of the output images.
#IMAGE_SIZE = (12, 8)

In [8]:
def run_inference_for_single_image(image, graph):
  with graph.as_default():
    with tf.Session() as sess:
      # Get handles to input and output tensors
      ops = tf.get_default_graph().get_operations()
      all_tensor_names = {output.name for op in ops for output in op.outputs}
      tensor_dict = {}
      for key in [
          'num_detections', 'detection_boxes', 'detection_scores',
          'detection_classes', 'detection_masks'
      ]:
        tensor_name = key + ':0'
        if tensor_name in all_tensor_names:
          tensor_dict[key] = tf.get_default_graph().get_tensor_by_name(
              tensor_name)
      if 'detection_masks' in tensor_dict:
        # The following processing is only for single image
        detection_boxes = tf.squeeze(tensor_dict['detection_boxes'], [0])
        detection_masks = tf.squeeze(tensor_dict['detection_masks'], [0])
        # Reframe is required to translate mask from box coordinates to image coordinates and fit the image size.
        real_num_detection = tf.cast(tensor_dict['num_detections'][0], tf.int32)
        detection_boxes = tf.slice(detection_boxes, [0, 0], [real_num_detection, -1])
        detection_masks = tf.slice(detection_masks, [0, 0, 0], [real_num_detection, -1, -1])
        detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
            detection_masks, detection_boxes, image.shape[0], image.shape[1])
        detection_masks_reframed = tf.cast(
            tf.greater(detection_masks_reframed, 0.5), tf.uint8)
        # Follow the convention by adding back the batch dimension
        tensor_dict['detection_masks'] = tf.expand_dims(
            detection_masks_reframed, 0)
      image_tensor = tf.get_default_graph().get_tensor_by_name('image_tensor:0')

      # Run inference
      output_dict = sess.run(tensor_dict,
                             feed_dict={image_tensor: np.expand_dims(image, 0)})

      # all outputs are float32 numpy arrays, so convert types as appropriate
      output_dict['num_detections'] = int(output_dict['num_detections'][0])
      output_dict['detection_classes'] = output_dict[
          'detection_classes'][0].astype(np.uint8)
      output_dict['detection_boxes'] = output_dict['detection_boxes'][0]
      output_dict['detection_scores'] = output_dict['detection_scores'][0]
      if 'detection_masks' in output_dict:
        output_dict['detection_masks'] = output_dict['detection_masks'][0]
  return output_dict

In [9]:
import os
import time
start_time = time.time()

CPU_Pct=str(round(float(os.popen('''grep 'cpu ' /proc/stat | awk '{usage=($2+$4)*100/($2+$4+$5)} END {print usage }' ''').readline()),2))
print('\x1b[1;38m'+"CPU Usage in begenning = " + CPU_Pct+"%"+'\x1b[0m')

#from __future__ import print_function



print ('\x1b[1;32m'+"person detector took time", time.time() - start_time, " seconds to open"+'\x1b[0m')
while True:
    url_response = urllib.request.urlopen(url)
    img_array = np.array(bytearray(url_response.read()), dtype=np.uint8)
    img = cv2.imdecode(img_array, -1)    
  #ret, image_np = cap.read()
  # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
    image_np_expanded = np.expand_dims(img, axis=0)
  # Actual detection.
    output_dict = run_inference_for_single_image(img, detection_graph)
    
    
    
  # Visualization of the results of a detection.
    vis_util.visualize_boxes_and_labels_on_image_array(
        img,
        output_dict['detection_boxes'],
        output_dict['detection_classes'],
        output_dict['detection_scores'],
        category_index,
        instance_masks=output_dict.get('detection_masks'),
        use_normalized_coordinates=True,
        line_thickness=2)
 

   # count = [category_index.get(value) for index,value in enumerate(classes[0]) if scores[0,index] > 0.5]
    #print('Number of Detected Person Is--> ', + len(count))
    print ('\x1b[1;31m'+"person detection in", time.time() - start_time,"th seconds "+'\x1b[0m')
    cv2.imshow('person detection', cv2.resize(img, (700,600)))
    
    CPU_Pct=str(round(float(os.popen('''grep 'cpu ' /proc/stat | awk '{usage=($2+$4)*100/($2+$4+$5)} END {print usage }' ''').readline()),2))
    print('\x1b[1;39m'+"CPU Usage currently = " + CPU_Pct+"%"+'\x1b[0m')
    
    import psutil
    #print(psutil.cpu_percent())
    print(psutil.virtual_memory())  # physical memory usage
    print('memory % used:', psutil.virtual_memory()[2])

    
    time.sleep(2)
    if cv2.waitKey(20) & 0xFF == ord('q'):
        break
        cv2.destroyAllWindows()
        
        
        
        
print ('\x1b[1;35m'+"person detection total time taken", time.time() - start_time, " seconds"+'\x1b[0m')

CPU Usage in begenning = 18.72%
person detector took time 0.02425241470336914  seconds to open
person detection in 7.416622877120972 th seconds 
CPU Usage currently = 18.74%
svmem(total=8244260864, available=3677491200, percent=55.4, used=3819409408, free=1796571136, active=4090937344, inactive=1902612480, buffers=231333888, cached=2396946432, shared=446173184, slab=264540160)
memory % used: 55.4
person detection in 13.740208864212036 th seconds 
CPU Usage currently = 18.76%
svmem(total=8244260864, available=3642941440, percent=55.8, used=3852357632, free=1761521664, active=4120449024, inactive=1904918528, buffers=231350272, cached=2399031296, shared=448049152, slab=264790016)
memory % used: 55.8
person detection in 19.275083303451538 th seconds 
CPU Usage currently = 18.77%
svmem(total=8244260864, available=3643875328, percent=55.8, used=3846533120, free=1762025472, active=4114501632, inactive=1910108160, buffers=231358464, cached=2404343808, shared=452878336, slab=265084928)
memory %